In [1]:
from glob import glob

In [2]:
ignore = ['t1']
stripfiles = sorted(glob('meshes/*strip*.obj'))
stripfiles = [fl for fl in stripfiles if 't1' not in fl]
print('    ', len(stripfiles))
print(stripfiles[0])
print(stripfiles[-1])

     100
meshes\frame98000_strip_t3_1154.obj
meshes\frame98000_strip_t6_766.obj


In [3]:
def indadd(istr, amount):
    parts = [str(int(val) + amount) for val in istr.split()[1:]]
    return 'f ' + ' '.join(parts)

print(indadd('f 1 2 3 ', 10))

def norm(ww, vert):
    coords = [float(val) / float(ww.split('# w ')[1]) for val in vert.split()[1:]]
    return 'v ' + ' '.join([str(val) for val in coords])

print(norm(*('# w 2', 'v 2 2 2')))

f 11 12 13
v 1.0 1.0 1.0


In [4]:
verts = []
inds = []
verts_normed = []
verts_og = []

voffset = 0
FROM =0
UPTO = 150
for fname in stripfiles[FROM:UPTO]:
    with open(fname) as fl:
        raw = fl.read()
    lines = raw.split('\n')
    lines = [ll for ll in lines if ll]
    vs = [ll for ll in lines if ll[0] == 'v']
    
    ws = [ll for ll in lines if ll[:3] == '# w']
    os = [ll.replace('# o', 'v') for ll in lines if ll[:3] == '# o']
    fs = [ll for ll in lines if ll[0] == 'f']
    print('%s: %d verts, %d faces, %d ws' % (fname, len(vs), len(fs), len(ws)))
    
    verts_normed += [norm(*pair) for pair in zip(ws, vs)]
    verts_og += os
    verts += ['# ' + fname]
    verts += vs
    inds += [indadd(fdef, voffset) for fdef in fs]
    voffset += len(vs)

print('Total:  %d verts,  %d faces' % (len(verts), len(inds)))

meshes\frame98000_strip_t3_1154.obj: 772 verts, 1152 faces, 772 ws
meshes\frame98000_strip_t3_1274.obj: 783 verts, 1272 faces, 783 ws
meshes\frame98000_strip_t3_1286.obj: 860 verts, 1284 faces, 860 ws
meshes\frame98000_strip_t3_1292.obj: 864 verts, 1290 faces, 864 ws
meshes\frame98000_strip_t3_1338.obj: 849 verts, 1336 faces, 849 ws
meshes\frame98000_strip_t3_1462.obj: 946 verts, 1460 faces, 946 ws
meshes\frame98000_strip_t3_1616.obj: 1080 verts, 1614 faces, 1080 ws
meshes\frame98000_strip_t3_164.obj: 112 verts, 162 faces, 112 ws
meshes\frame98000_strip_t3_176.obj: 120 verts, 174 faces, 120 ws
meshes\frame98000_strip_t3_1852.obj: 1228 verts, 1850 faces, 1228 ws
meshes\frame98000_strip_t3_194.obj: 132 verts, 192 faces, 132 ws
meshes\frame98000_strip_t3_1952.obj: 1242 verts, 1950 faces, 1242 ws
meshes\frame98000_strip_t3_2002.obj: 1311 verts, 2000 faces, 1311 ws
meshes\frame98000_strip_t3_206.obj: 140 verts, 204 faces, 140 ws
meshes\frame98000_strip_t3_212.obj: 144 verts, 210 faces, 144 

In [5]:
with open('dump.obj', 'w') as fl:
    fl.write('\n'.join(verts))
    fl.write('\n')
    fl.write('\n'.join(inds))

In [ ]:
# with open('dump_norm.obj', 'w') as fl:
    fl.write('\n'.join(verts_normed))
    fl.write('\n')
    fl.write('\n'.join(inds))

In [6]:
with open('dump_og.obj', 'w') as fl:
    fl.write('\n'.join(verts_og))
    fl.write('\n')
    fl.write('\n'.join(inds))